In [1]:
MODEL_NAME = "gpt2-xl"  # gpt2-{medium,large,xl} or EleutherAI/gpt-j-6B
MODEL_NAME = "/mnt/workspace/guoyiqiu/coding/vicuna_7b"

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
model, tok = (
    AutoModelForCausalLM.from_pretrained(MODEL_NAME),
    AutoTokenizer.from_pretrained(MODEL_NAME),
)
if "llama" in model.__class__.__name__.lower():
    print("loading llama model with half precision")
    model.half()
    
    if not tok.eos_token:
        print("adding eos/bos/unk/ token to tokenizer")
        tok.add_special_tokens({
            "eos_token": "</s>",
            "bos_token": "<s>",
            "unk_token": "<unk>",
        })
tok.padding_side = "right"
model.cuda()
tok.pad_token = tok.eos_token
orig_weights = None

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

loading llama model with half precision
adding eos/bos/unk/ token to tokenizer


In [3]:
import unicodedata
from typing import List, Optional

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
import os
from pathlib import Path
from typing import Dict, List, Tuple
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from rome import ROMEHyperParams, apply_rome_to_model
from util import nethook
from util.globals import *
from copy import deepcopy
from typing import Dict, List, Tuple

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer



CONTEXT_TEMPLATES_CACHE = None
import os
from pathlib import Path
from typing import Dict, List

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

from rome import repr_tools
from util.globals import *
# Cache variables

from typing import Dict, List, Tuple

import numpy as np
import torch
from matplotlib.style import context
from transformers import AutoModelForCausalLM, AutoTokenizer

from rome import repr_tools
from util import nethook
import os
from pathlib import Path

import torch
from datasets import load_dataset
from tqdm.auto import tqdm
from transformers import AutoModelForCausalLM, AutoTokenizer

from util.globals import *
from util.nethook import Trace, set_requires_grad
from util.runningstats import CombinedStat, Mean, NormMean, SecondMoment, tally

from rome.tok_dataset import (
    TokenizedDataset,
    dict_to_,
    flatten_masked_batch,
    length_collation,
)
inv_mom2_cache = {}
STAT_TYPES = {
    "mom2": SecondMoment,
    "mean": Mean,
    "norm_mean": NormMean,
}

from copy import deepcopy
from typing import List

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

from util import nethook

rbracket_replace = lambda x, y : x[:x.rfind("{}")] + y + x[x.rfind("{}")+2:]

def get_reprs_at_word_tokens(
    model: AutoModelForCausalLM,
    tok: AutoTokenizer,
    context_templates: List[str],
    words: List[str],
    layer: int,
    module_template: str,
    subtoken: str,
    track: str = "in",
) -> torch.Tensor:
    """
    Retrieves the last token representation of `word` in `context_template`
    when `word` is substituted into `context_template`. See `get_last_word_idx_in_template`
    for more details.
    """

    idxs = get_words_idxs_in_templates(tok, context_templates, words, subtoken)
    return get_reprs_at_idxs(
        model,
        tok,
        # [context_templates[i].format(words[i]) for i in range(len(words))],
        [rbracket_replace(context_templates[i], words[i]) for i in range(len(words))],
        idxs,
        layer,
        module_template,
        track,
    )


def get_words_idxs_in_templates(
    tok: AutoTokenizer, context_templates: str, words: str, subtoken: str
) -> int:
    """
    Given list of template strings, each with *one* format specifier
    (e.g. "{} plays basketball"), and words to be substituted into the
    template, computes the post-tokenization index of their last tokens.
    """

    assert all(
        tmp.count("{}") == 1 for tmp in context_templates
    ), "We currently do not support multiple fill-ins for context"

    # Compute prefixes and suffixes of the tokenized context
    fill_idxs = [tmp.index("{}") for tmp in context_templates]
    prefixes, suffixes = [
        tmp[: fill_idxs[i]] for i, tmp in enumerate(context_templates)
    ], [tmp[fill_idxs[i] + 2 :] for i, tmp in enumerate(context_templates)]
    words = deepcopy(words)

    # Pre-process tokens
    for i, prefix in enumerate(prefixes):
        if len(prefix) > 0:
            assert prefix[-1] == " "
            prefix = prefix[:-1]

            prefixes[i] = prefix
            words[i] = f" {words[i].strip()}"
        elif "llama" in tok.__class__.__name__.lower():
            words[i] = f" {words[i].strip()}"

    # Tokenize to determine lengths
    assert len(prefixes) == len(words) == len(suffixes)
    n = len(prefixes)
    batch_tok = tok([*prefixes, *words, *suffixes])
    prefixes_tok, words_tok, suffixes_tok = [
        batch_tok[i : i + n] for i in range(0, n * 3, n)
    ]
    prefixes_len, words_len, suffixes_len = [
        [len(el) for el in tok_list]
        for tok_list in [prefixes_tok, words_tok, suffixes_tok]
    ]
    
    if "llama" in tok.__class__.__name__.lower():
        words_len = [l-2 for l in words_len]

    # Compute indices of last tokens
    if subtoken == "last" or subtoken == "first_after_last":
        return [
            [
                prefixes_len[i]
                + words_len[i]
                - (1 if subtoken == "last" or suffixes_len[i] == 0 else 0)
            ]
            # If suffix is empty, there is no "first token after the last".
            # So, just return the last token of the word.
            for i in range(n)
        ]
    elif subtoken == "first":
        return [[prefixes_len[i]] for i in range(n)]
    else:
        raise ValueError(f"Unknown subtoken type: {subtoken}")


def get_reprs_at_idxs(
    model: AutoModelForCausalLM,
    tok: AutoTokenizer,
    contexts: List[str],
    idxs: List[List[int]],
    layer: int,
    module_template: str,
    track: str = "in",
) -> torch.Tensor:
    """
    Runs input through model and returns averaged representations of the tokens
    at each index in `idxs`.
    """

    def _batch(n):
        for i in range(0, len(contexts), n):
            yield contexts[i : i + n], idxs[i : i + n]

    assert track in {"in", "out", "both"}
    both = track == "both"
    tin, tout = (
        (track == "in" or both),
        (track == "out" or both),
    )
    module_name = module_template.format(layer)
    to_return = {"in": [], "out": []}

    def _process(cur_repr, batch_idxs, key):
        nonlocal to_return
        cur_repr = cur_repr[0] if type(cur_repr) is tuple else cur_repr
        for i, idx_list in enumerate(batch_idxs):
            to_return[key].append(cur_repr[i][idx_list].mean(0))

    for batch_contexts, batch_idxs in _batch(n=512):
        contexts_tok = tok(batch_contexts, padding=True, return_tensors="pt").to(
            next(model.parameters()).device
        )
        input_ids = contexts_tok["input_ids"]
        attention_mask = contexts_tok["attention_mask"]
        with torch.no_grad():
            with nethook.Trace(
                module=model,
                layer=module_name,
                retain_input=tin,
                retain_output=tout,
            ) as tr:
                model(input_ids=input_ids, attention_mask=attention_mask)

        if tin:
            _process(tr.input, batch_idxs, "in")
        if tout:
            _process(tr.output, batch_idxs, "out")

    to_return = {k: torch.stack(v, 0) for k, v in to_return.items() if len(v) > 0}

    if len(to_return) == 1:
        return to_return["in"] if tin else to_return["out"]
    else:
        return to_return["in"], to_return["out"]


def layer_stats(
    model,
    tokenizer,
    layer_name,
    stats_dir,
    ds_name,
    to_collect,
    model_name=None,
    sample_size=None,
    precision=None,
    batch_tokens=None,
    download=True,
    progress=tqdm,
):
    """
    Function to load or compute cached stats.
    """

    def get_ds():
        raw_ds = load_dataset(
            ds_name,
            dict(wikitext="wikitext-103-raw-v1", wikipedia="20200501.en")[ds_name],
        )
        maxlen = model.config.n_positions
        if batch_tokens is not None and batch_tokens < maxlen:
            maxlen = batch_tokens
        return TokenizedDataset(raw_ds["train"], tokenizer, maxlen=maxlen)

    # Continue with computation of statistics
    batch_size = 100  # Examine this many dataset texts at once
    npos = model.config.n_positions
    if batch_tokens is None:
        batch_tokens = npos * 3  # Sort and divide into batches with this many tokens
    if precision is None:
        precision = "float64"
    dtype = getattr(torch, precision)
    size_suffix = "" if sample_size is None else f"_{sample_size}"
    if batch_tokens < npos:
        size_suffix = "_t{batch_tokens}" + size_suffix
    if model_name is None:
        model_name = model.config._name_or_path.replace("/", "_")

    stats_dir = Path(stats_dir)
    file_extension = f"{model_name}/{ds_name}_stats/{layer_name}_{precision}_{'-'.join(sorted(to_collect))}{size_suffix}.npz"
    filename = stats_dir / file_extension

    if not filename.exists() and download:
        remote_url = f"{REMOTE_ROOT_URL}/data/stats/{file_extension}"
        try:
            print(f"Attempting to download {file_extension} from {remote_url}.")
            (stats_dir / "/".join(file_extension.split("/")[:-1])).mkdir(
                exist_ok=True, parents=True
            )
            torch.hub.download_url_to_file(remote_url, filename)
            print("Successfully downloaded.")
        except Exception as e:
            print(f"Unable to download due to {e}. Computing locally....")

    ds = get_ds() if not filename.exists() else None

    if progress is None:
        progress = lambda x: x

    stat = CombinedStat(**{k: STAT_TYPES[k]() for k in to_collect})
    loader = tally(
        stat,
        ds,
        cache=filename,
        sample_size=sample_size,
        batch_size=batch_size,
        collate_fn=length_collation(batch_tokens),
        pin_memory=True,
        random_sample=1,
        num_workers=2,
    )
    batch_count = -(-(sample_size or len(ds)) // batch_size)
    with torch.no_grad():
        for batch_group in progress(loader, total=batch_count):
            for batch in batch_group:
                batch = dict_to_(batch, "cuda")
                with Trace(
                    model, layer_name, retain_input=True, retain_output=False, stop=True
                ) as tr:
                    model(**batch)
                feats = flatten_masked_batch(tr.input, batch["attention_mask"])
                # feats = flatten_masked_batch(tr.output, batch["attention_mask"])
                feats = feats.to(dtype=dtype)
                stat.add(feats)
    return stat


def compute_v(
    model: AutoModelForCausalLM,
    tok: AutoTokenizer,
    request: Dict,
    hparams: ROMEHyperParams,
    layer: int,
    left_vector: torch.Tensor,
    context_templates: List[str],
) -> torch.Tensor:
    """
    Computes the value (right) vector for the rank-1 update.
    Runs a simple optimization procedure.
    """

    print("Computing right vector (v)")

    # Tokenize target into list of int token IDs
    target_ids = tok(request["target_new"]["str"], return_tensors="pt").to("cuda")["input_ids"][0]
    
    if "llama" in tok.__class__.__name__.lower():
        print("detecting llama tokenizer, strip input and cut bos token")
        target_ids = tok(request["target_new"]["str"].strip(), return_tensors="pt").to("cuda")["input_ids"][0,1:]
    
    print(f"target_ids:{target_ids} target_tokens:{tok.batch_decode(target_ids)}")
   
    # Compile list of rewriting and KL x/y pairs
    rewriting_prompts = [
        (rbracket_replace(context, request["prompt"]) + " " + tok.decode(target_ids[:-1])).strip() # 如果是llama tokenizer，对数字要在前面加空格
        for context in context_templates
    ]
    kl_prompts = ["{} is a"]
    all_prompts = rewriting_prompts + kl_prompts
    
    print(f"all_prompts:{all_prompts}")
    
    input_tok = tok(
        [rbracket_replace(prompt, request["subject"]) for prompt in all_prompts],
        return_tensors="pt",
        padding=True,
    ).to("cuda")
    
    print(f"input_tok: {[tok.batch_decode(i)for i in input_tok['input_ids']]}")
    
    # Compute rewriting targets
    rewriting_targets = torch.tensor(-100, device="cuda").repeat(
        len(rewriting_prompts), *input_tok["input_ids"].shape[1:]
    )
    
    for i in range(len(rewriting_prompts)):
        ex_len = input_tok["attention_mask"][i].sum()
        rewriting_targets[i, ex_len - len(target_ids) : ex_len] = target_ids
    
    print(f"rewriting_targets shape: {rewriting_targets.shape}")
    
    # Compute indices of the tokens where the fact is looked up
    lookup_idxs = [
        find_fact_lookup_idx(
            prompt, request["subject"], tok, hparams.fact_token, verbose=(i==0)
        )
        for i, prompt in enumerate(all_prompts)
    ]

    print(f"lookup_idxs[0]: {lookup_idxs[0]} rewriting_targets[0]:{rewriting_targets[0]}")
    
    # Finalize rewrite and loss layers
    loss_layer = max(hparams.v_loss_layer, layer)
    print(f"Rewrite layer is {layer}")
    print(f"Tying optimization objective to {loss_layer}")

    # Set up an optimization over a latent vector that, when output at the
    # rewrite layer, i.e. hypothesized fact lookup location, will induce the
    # target token to be predicted at the final layer.
    delta = torch.zeros((model.config.hidden_size,), requires_grad=True, device="cuda")
    target_init, kl_distr_init = None, None

    # Inserts new "delta" variable at the appropriate part of the computation
    def edit_output_fn(cur_out, cur_layer):
        nonlocal target_init

        if cur_layer == hparams.mlp_module_tmp.format(layer):
            # Store initial value of the vector of interest
            if target_init is None:
                print("Recording initial value of v*")
                # Initial value is recorded for the clean sentence
                target_init = cur_out[0, lookup_idxs[0]].detach().clone()

            for i, idx in enumerate(lookup_idxs):
                cur_out[i, idx, :] += delta

        return cur_out

    # Optimizer
    opt = torch.optim.Adam([delta], lr=hparams.v_lr)
    nethook.set_requires_grad(False, model)

    # Execute optimization
    for it in range(hparams.v_num_grad_steps):
        opt.zero_grad()

        # Forward propagation
        with nethook.TraceDict(
            module=model,
            layers=[
                hparams.layer_module_tmp.format(loss_layer),
                hparams.mlp_module_tmp.format(layer),
            ],
            retain_input=False,
            retain_output=True,
            edit_output=edit_output_fn,
        ) as tr:
            logits = model(input_ids=input_tok["input_ids"], attention_mask=input_tok["attention_mask"]).logits

            # Compute distribution for KL divergence
            kl_logits = torch.stack(
                [
                    logits[i - len(kl_prompts), lookup_idx, :]
                    for i, lookup_idx in enumerate(lookup_idxs[-len(kl_prompts) :])
                ],
                dim=0,
            )
            kl_log_probs = torch.nn.functional.log_softmax(kl_logits, dim=1)
            if kl_distr_init is None:
                kl_distr_init = kl_log_probs.detach().clone()

        # Compute loss on rewriting targets
        log_probs = torch.log_softmax(logits, dim=2) # [batch_size, seq_len, vocab_size]

        loss = torch.gather(
            log_probs,
            2,
            torch.where(rewriting_targets != -100, rewriting_targets, 0).unsqueeze(2), # [batch_size, seq_len, 1]
        ).squeeze(2) # [batch_size, seq_len] 把log_probs中的rewriting_targets的概率取出来
        mask = (rewriting_targets != -100).float() # [batch_size, seq_len]

        # Aggregate total losses
        nll_loss_each = -(loss * mask).sum(1) / target_ids.size(0) # 把所有rewriting_targets关于target_ids的loss取出来，并且除以target_ids的长度
        nll_loss = nll_loss_each.mean() # 取batch平均, nll_loss就是所有rewrite_prompts上，rewriting_targets
        kl_loss = hparams.kl_factor * torch.nn.functional.kl_div(
            kl_distr_init, kl_log_probs, log_target=True, reduction="batchmean"
        )
        weight_decay = hparams.v_weight_decay * (
            torch.norm(delta) / torch.norm(target_init) ** 2
        )
        loss = nll_loss + kl_loss + weight_decay
        print(
            f"loss {np.round(loss.item(), 3)} = {np.round(nll_loss.item(), 3)} + {np.round(kl_loss.item(), 3)} + {np.round(weight_decay.item(), 3)} "
            f"avg prob of [{request['target_new']['str']}] "
            f"{torch.exp(-nll_loss_each).mean().item()}"
        )
        if loss < 5e-2:
            break

        if it == hparams.v_num_grad_steps - 1:
            break

        # Backpropagate
        loss.backward()
        opt.step()

        # Project within L2 ball
        max_norm = hparams.clamp_norm_factor * target_init.norm()
        if delta.norm() > max_norm:
            with torch.no_grad():
                delta[...] = delta * max_norm / delta.norm()

    target = target_init + delta

    # Retrieve cur_input, the current input to the 2nd MLP layer, and
    # cur_output, the original output of the 2nd MLP layer.
    cur_input, cur_output = get_module_input_output_at_word(
        model,
        tok,
        layer,
        context_template=request["prompt"],
        word=request["subject"],
        module_template=hparams.rewrite_module_tmp,
        fact_token_strategy=hparams.fact_token,
    )

    # Solving the linear system to compute the right vector
    right_vector = (target - cur_output) / torch.dot(cur_input, left_vector)
    print(f"Delta norm: {(target - cur_output).norm().item()}")
    print(
        f"Change in target norm: {target_init.norm().item()} to {target.norm().item()} => {(target.norm() - target_init.norm()).item()}"
    )
    print(f"Division Factor: {torch.dot(cur_input, left_vector).item()}")
    print(f"Right vector norm: {right_vector.norm()}")

    return right_vector


def get_module_input_output_at_word(
    model: AutoModelForCausalLM,
    tok: AutoTokenizer,
    layer: int,
    context_template: str,
    word: str,
    module_template: str,
    fact_token_strategy: str,
) -> Tuple[torch.Tensor]:
    """
    Retrieves detached representations for a word at the input and
    output of a particular layer module.
    """

    word_repr_args = dict(
        model=model,
        tok=tok,
        layer=layer,
        module_template=module_template,
    )
    if "subject_" in fact_token_strategy and fact_token_strategy.index("subject_") == 0:
        subtoken = fact_token_strategy[len("subject_") :]
        l_input, l_output = get_reprs_at_word_tokens(
            track="both",
            subtoken=subtoken,
            context_templates=[context_template],
            words=[word],
            **word_repr_args,
        )
    elif fact_token_strategy == "last":
        l_input, l_output = get_reprs_at_idxs(
            track="both",
            contexts=[context_template.format(word)],
            idxs=[[-1]],
            **word_repr_args,
        )
    else:
        raise ValueError(f"fact_token={fact_token_strategy} not recognized")

    l_input, l_output = l_input[0], l_output[0]
    return l_input.detach(), l_output.detach()


def find_fact_lookup_idx(
    prompt: str,
    subject: str,
    tok: AutoTokenizer,
    fact_token_strategy: str,
    verbose=True,
) -> int:
    """
    Computes hypothesized fact lookup index given a sentence and subject.
    """

    ret = None
    if fact_token_strategy == "last":
        ret = -1
    elif (
        "subject_" in fact_token_strategy and fact_token_strategy.index("subject_") == 0
    ):  
        ret = get_words_idxs_in_templates(
            tok=tok,
            context_templates=[prompt],
            words=[subject],# 这里的subject是没有空格的
            subtoken=fact_token_strategy[len("subject_") :],
        )[0][0]
    else:
        raise ValueError(f"fact_token={fact_token_strategy} not recognized")

    sentence = rbracket_replace(prompt, subject)
    if verbose:
        print(
            f"Lookup index found: {ret} | Sentence: {sentence} | Token:",
            tok.decode(tok(sentence)["input_ids"][ret]),
        )

    return ret


def get_inv_cov(
    model: AutoModelForCausalLM,
    tok: AutoTokenizer,
    layer_name: str,
    mom2_dataset: str,
    mom2_n_samples: str,
    mom2_dtype: str,
) -> torch.Tensor:
    """
    Retrieves covariance statistics, then computes the algebraic inverse.
    Caches result for future use.
    """

    global inv_mom2_cache

    model_name = model.config._name_or_path.replace("/", "_")
    key = (model_name, layer_name)

    if key not in inv_mom2_cache:
        print(
            f"Retrieving inverse covariance statistics for {model_name} @ {layer_name}. "
            f"The result will be cached to avoid repetitive computation."
        )
        stat = layer_stats(
            model,
            tok,
            layer_name,
            STATS_DIR,
            mom2_dataset,
            to_collect=["mom2"],
            sample_size=mom2_n_samples,
            precision=mom2_dtype,
        )
        inv_mom2_cache[key] = torch.inverse(
            stat.mom2.moment().to("cuda")
        ).float()  # Cast back to float32

    return inv_mom2_cache[key]


def compute_u(
    model: AutoModelForCausalLM,
    tok: AutoTokenizer,
    request: Dict,
    hparams: ROMEHyperParams,
    layer: int,
    context_templates: List[str],
) -> torch.Tensor:
    """
    Computes the right vector used in constructing the rank-1 update matrix.
    """

    print("Computing left vector (u)...")

    # Compute projection token
    word_repr_args = dict(
        model=model,
        tok=tok,
        layer=layer,
        module_template=hparams.rewrite_module_tmp,
        track="in",
    )
    if "subject_" in hparams.fact_token and hparams.fact_token.index("subject_") == 0:
        word = request["subject"]
        print(f"Selected u projection object {word}")
        cur_repr = get_reprs_at_word_tokens(
            context_templates=[
                # templ.format(request["prompt"]) for templ in context_templates
                rbracket_replace(templ, request["prompt"]) for templ in context_templates
                ],
            words=[word for _ in range(len(context_templates))],
            subtoken=hparams.fact_token[len("subject_") :],
            **word_repr_args,
        ).mean(0)
    elif hparams.fact_token == "last":
        # Heuristic to choose last word. Not a huge deal if there's a minor
        # edge case (e.g. multi-token word) because the function below will
        # take the last token.
        cur_repr = get_reprs_at_idxs(
            contexts=[
                templ.format(request["prompt"].format(request["subject"]))
                for templ in context_templates
            ],
            idxs=[[-1] for _ in range(len(context_templates))],
            **word_repr_args,
        ).mean(0)
        print("Selected u projection token with last token")
    else:
        raise ValueError(f"fact_token={hparams.fact_token} not recognized")

    # Apply inverse second moment adjustment
    u = cur_repr
    if hparams.mom2_adjustment:
        u = get_inv_cov(
            model,
            tok,
            hparams.rewrite_module_tmp.format(layer),
            hparams.mom2_dataset,
            hparams.mom2_n_samples,
            hparams.mom2_dtype,
        ).float() @ u.unsqueeze(1).float()
        u = u.squeeze()

    return u / u.norm()


def apply_rome_to_model(
    model: AutoModelForCausalLM,
    tok: AutoTokenizer,
    requests: List[Dict],
    hparams: ROMEHyperParams,
    copy=False,
    return_orig_weights=False,
) -> Tuple[AutoModelForCausalLM, List[str]]:
    """
    Returns a model with the desired changes.

    :param copy: If true, will preserve the original model while creating a new one to edit.
        Note that you are responsible for deallocating the new model's memory to avoid leaks.

    :return: (1) the updated model, (2) an original copy of the weights that changed
    """

    if copy:
        model = deepcopy(model)

    weights_copy = {}

    for i, request in enumerate(requests):
        deltas = execute_rome(model, tok, request, hparams)

        with torch.no_grad():
            for w_name, (delta_u, delta_v) in deltas.items():
                upd_matrix = delta_u.unsqueeze(1).float() @ delta_v.unsqueeze(0).float()
                w = nethook.get_parameter(model, w_name)
                upd_matrix = upd_matrix_match_shape(upd_matrix, w.shape)

                if return_orig_weights and w_name not in weights_copy:
                    assert i == 0
                    weights_copy[w_name] = w.detach().clone()

                w[...] += upd_matrix

        print(f"New weights successfully inserted into {w} {list(deltas.keys())}")

    return model, weights_copy


def execute_rome(
    model: AutoModelForCausalLM,
    tok: AutoTokenizer,
    request: Dict,
    hparams: ROMEHyperParams,
) -> Dict[str, Tuple[torch.Tensor]]:
    """
    Executes the ROME update algorithm for the specified update at the specified layer
    Invariant: model at beginning of function == model at end of function
    """

    # Update target and print info
    request = deepcopy(request)
    if request["target_new"]["str"][0] != " ":
        # Space required for correct tokenization
        request["target_new"]["str"] = " " + request["target_new"]["str"]
    print(
        f"Executing ROME algorithm for the update: "
        f"[{request['prompt'].format(request['subject'])}] -> [{request['target_new']['str']}]"
    )

    # Retrieve weights that user desires to change
    weights = {
        f"{hparams.rewrite_module_tmp.format(layer)}.weight": nethook.get_parameter(
            model, f"{hparams.rewrite_module_tmp.format(layer)}.weight"
        )
        for layer in hparams.layers
    }
    # Save old weights for future restoration
    weights_copy = {k: v.detach().clone() for k, v in weights.items()}

    # Update loop: sequentially intervene at each specified layer
    deltas = {}
    for layer in sorted(hparams.layers):
        # Compute rank-1 update matrix
        left_vector: torch.Tensor = compute_u(
            model,
            tok,
            request,
            hparams,
            layer,
            get_context_templates(model, tok, hparams.context_template_length_params),
        )
        print("Left vector shape:", left_vector.shape)
        right_vector: torch.Tensor = compute_v(
            model,
            tok,
            request,
            hparams,
            layer,
            left_vector,
            get_context_templates(model, tok, hparams.context_template_length_params),
        )
        print("Right vector shape:", right_vector.shape)

        with torch.no_grad():
            # Determine correct transposition of delta matrix
            weight_name = f"{hparams.rewrite_module_tmp.format(layer)}.weight"
            upd_matrix = left_vector.unsqueeze(1).float() @ right_vector.unsqueeze(0).float()
            upd_matrix = upd_matrix_match_shape(upd_matrix, weights[weight_name].shape)

            # Update model weights and record desired changes in `delta` variable
            weights[weight_name][...] += upd_matrix
            deltas[weight_name] = (
                left_vector.detach(),
                right_vector.detach(),
            )

    # Restore state of original model
    with torch.no_grad():
        for k, v in weights.items():
            v[...] = weights_copy[k]

    print(f"Deltas successfully computed for {list(weights.keys())}")

    return deltas


def upd_matrix_match_shape(matrix: torch.Tensor, shape: torch.Size) -> torch.Tensor:
    """
    GPT-2 and GPT-J have transposed weight representations.
    Returns a matrix that matches the desired shape, else raises a ValueError
    """

    if matrix.shape == shape:
        return matrix
    elif matrix.T.shape == shape:
        return matrix.T
    else:
        raise ValueError(
            "Update matrix computed by ROME does not match original weight shape. "
            "Check for bugs in the code?"
        )


def get_context_templates(model, tok, length_params):
    global CONTEXT_TEMPLATES_CACHE

    if CONTEXT_TEMPLATES_CACHE is None:
        CONTEXT_TEMPLATES_CACHE = ["{}"] + [
            x + ". {}"
            for x in sum(
                (
                    generate_fast(
                        model,
                        tok,
                        [tok.bos_token],
                        n_gen_per_prompt=n_gen,
                        max_out_len=length,
                    )
                    for length, n_gen in length_params
                ),
                [],
            )
        ]

        print(f"Cached context templates {CONTEXT_TEMPLATES_CACHE}")

    return CONTEXT_TEMPLATES_CACHE


def generate_fast(
    model: AutoModelForCausalLM,
    tok: AutoTokenizer,
    prompts: List[str],
    n_gen_per_prompt: int = 1,
    top_k: int = 5,
    max_out_len: int = 200,
    do_sample: bool = True,
):  
    orig_padding_side = tok.padding_side
    tok.padding_side = "left"
    inp = [prompt for prompt in prompts for _ in range(n_gen_per_prompt)]
    
    max_bsz = 8
    inps = [inp[i:i+max_bsz] for i in range(0, len(inp), max_bsz)]
    txts = []
    
    for inp in tqdm(inps,total=len(inps)):
        inp_tok = tok(inp, padding=True, return_tensors="pt").to(
            next(model.parameters()).device
        )
        input_ids, attention_mask = inp_tok["input_ids"], inp_tok["attention_mask"]
        output_ids = model.generate(input_ids=input_ids, attention_mask=attention_mask, max_length=max_out_len, do_sample=do_sample)
        txt = [tok.decode(x) for x in output_ids.detach().cpu().numpy().tolist()]
        txt = [
            unicodedata.normalize("NFKD", x)
            .replace("\n\n", " ")
            .replace("<|endoftext|>", "")
            .replace(tok.eos_token,"")
            .replace(tok.bos_token,"")
            .replace(tok.unk_token,"")
            .strip()
            for x in txt
        ]
        txts.extend(txt)
    
    tok.padding_side = orig_padding_side
    return txts


def demo_model_editing(
    model: AutoModelForCausalLM,
    tok: AutoTokenizer,
    requests: List[Dict],
    generation_prompts: List[str],
    alg_name: str = "ROME",
) -> Tuple[AutoModelForCausalLM, Dict[str, torch.Tensor]]:
    """
    Applies the selected model editing algorithm. Generates text both before and after
    for comparison of model behavior. Returns the updated model and the original values of
    weights that were changed.
    """

    nethook.set_requires_grad(True, model)

    RewritingParamsClass, apply_method, hparams_prefix, hparams_suffix = load_alg(
        alg_name
    )
    params_name = (
        HPARAMS_DIR
        / hparams_prefix
        / f"{model.config._name_or_path.replace('/', '_')}{hparams_suffix}.json"
    )

    print_loud(f"Retrieving {alg_name} hyperparameters")
    print("Loading from", params_name)
    hparams = RewritingParamsClass.from_json(params_name)
    print(hparams)

    print_loud("Generating pre-update text")
    pre_update_text = generate_fast(model, tok, generation_prompts, max_out_len=100, do_sample=False)
    print(pre_update_text)

    print_loud(f"Applying {alg_name} to model")
    model_new, orig_weights = apply_method(
        model, tok, requests, hparams, return_orig_weights=True
    )

    print_loud("Generating post-update text")
    post_update_text = generate_fast(
        model_new, tok, generation_prompts, max_out_len=100, do_sample=False
    )
    print(post_update_text)

    print_loud("Summarizing differences")
    f = open("rome_output.txt", "w")
    for i, (prompt, pre, post) in enumerate(
        zip(generation_prompts, pre_update_text, post_update_text)
    ):
        if i > 0:
            print("".join(["-" for _ in range(10)]))

        prompt_str = "[Prompt]:"
        pre_str = f"[Pre-{alg_name}]:"
        post_str = f"[Post-{alg_name}]:"
        pad_to = 1 + max(len(prompt_str), len(pre_str), len(post_str))
        
        for s, t in zip([prompt_str, post_str, pre_str], [prompt, post, pre]):
            print(s.ljust(pad_to), t)
            f.write(s.ljust(pad_to) + t + "\n")
    f.close()
    return model_new, orig_weights


def load_alg(alg_name):
    """
    Loads dependencies for the desired algorithm.
    Implementation is slightly awkward to prevent unnecessary imports on Colab.

    The return value is a tuple of the following:
    1. Class for storing hyperparameters
    2. Method for applying rewrites
    3. Location of parameters
    4. Predefined suffix for the param file
    """
    assert alg_name in [
        "FT",
        "FT-L",
        "FT-AttnEdit",
        "KN",
        "MEND",
        "MEND-CF",
        "MEND-zsRE",
        "KE",
        "KE-CF",
        "ROME",
    ]

    if alg_name == "ROME":
        return ROMEHyperParams, apply_rome_to_model, "ROME", ""
    elif "FT" in alg_name:
        d = {
            "FT": (FTHyperParams, apply_ft_to_model, "FT", "_unconstr"),
            "FT-AttnEdit": (FTHyperParams, apply_ft_to_model, "FT", "_attn"),
            "FT-L": (FTHyperParams, apply_ft_to_model, "FT", "_constr"),
        }
        return d[alg_name]
    else:
        from baselines.efk import EFKHyperParams, EfkRewriteExecutor
        from baselines.kn import KNHyperParams, apply_kn_to_model
        from baselines.mend import MENDHyperParams, MendRewriteExecutor

        d = {
            "KN": (KNHyperParams, apply_kn_to_model, "KN", ""),
            "MEND": (MENDHyperParams, MendRewriteExecutor().apply_to_model, "MEND", ""),
            "KE": (EFKHyperParams, EfkRewriteExecutor().apply_to_model, "KE", ""),
            "MEND-CF": (
                MENDHyperParams,
                MendRewriteExecutor().apply_to_model,
                "MEND",
                "_CF",
            ),
            "MEND-zsRE": (
                MENDHyperParams,
                MendRewriteExecutor().apply_to_model,
                "MEND",
                "_zsRE",
            ),
            "KE-CF": (
                EFKHyperParams,
                EfkRewriteExecutor().apply_to_model,
                "MEND",
                "_CF",
            ),
        }
        return d[alg_name]


def print_loud(x, pad=3):
    """
    Prints a string with # box for emphasis.

    Example:
    ############################
    #                          #
    #  Applying ROME to model  #
    #                          #
    ############################
    """

    n = len(x)
    print()
    print("".join(["#" for _ in range(n + 2 * pad)]))
    print("#" + "".join([" " for _ in range(n + 2 * (pad - 1))]) + "#")
    print(
        "#"
        + "".join([" " for _ in range(pad - 1)])
        + x
        + "".join([" " for _ in range(pad - 1)])
        + "#"
    )
    print("#" + "".join([" " for _ in range(n + 2 * (pad - 1))]) + "#")
    print("".join(["#" for _ in range(n + 2 * pad)]))


In [4]:
import os
import sys
import time
import pytorch_lightning as pl
import torch
sys.path.append('/mnt/workspace/guoyiqiu/coding/gpt_re')
from model import *
import torch.utils.data as tud
from torch.utils.data import DataLoader
from lightning.pytorch.loggers import WandbLogger
from tqdm.notebook import tqdm
from utils.my_utils import *
import torch.nn.functional as F
import random
import regex as re
from dataset import *
import ipywidgets as widgets
from IPython.display import display
from typing import Union, List
pl.seed_everything(42)
torch.set_float32_matmul_precision('medium')
os.environ['TOKENIZERS_PARALLELISM'] = 'true'
os.environ['CUDA_VISIBLE_DEVICES'] = '3'

model_list = [
    ("gpt2", "/mnt/workspace/guoyiqiu/coding/huggingface/hub/models--gpt2/snapshots/e7da7f221d5bf496a48136c0cd264e630fe9fcc8"),
    ("gpt2-xl", "/mnt/workspace/guoyiqiu/coding/huggingface/hub/models--gpt2-xl/snapshots/33cdb5c0db5423c1879b1b9f16c352988e8754a8"),
    ("llama_7b", "/nvme/share/guoyiqiu/llama-7b"),
    ("llama_13b", "/nvme/share/guoyiqiu/llama-13b"),
    ("vicuna_7b", "/mnt/workspace/guoyiqiu/coding/vicuna_7b"),
    ("vicuna_13b", "/mnt/workspace/guoyiqiu/coding/vicuna-13b-v1.1"),
    ("book_7b", "/mnt/workspace/guoyiqiu/coding/Book_7B/checkpoint-4968"),
]


def setup_widgets(model_list):
    global mt_dropdown
    global setup_btn
    global device_tbtn
    global precision_tbtn
    global mnt_slider
    global input_textarea
    global output_textarea
    global submit_btn

    def setup_llm(btn):
        global mt
        global vis
        time_st = time.time()
        btn.description = "Loading model..."
        mt = LLM.from_pretrained(model_name=mt_dropdown.value, fp16=(precision_tbtn.value == "half"),)
        btn.description = "setup FlowVisualizer..."
        vis = FlowVisualizer(mt)
        btn.description = "Everything is ready."
        device_tbtn.value = 'cpu'
        print(f"Time cost: {time.time() - time_st:.2f}s")
    
    def switch_device(change):
        device_tbtn.disabled = True
        mt.to(change.new)
        torch.cuda.empty_cache() if change.new == 'cpu' else None
        device_tbtn.disabled = False

    def switch_precision(change):
        precision_tbtn.disabled = True
        if mt is not None:
            mt.model = mt.model.half() if change.new == 'half' else mt.model.float()
        precision_tbtn.disabled = False

    def generate(btn):
        btn.disabled = True
        submit_btn.description = "Generating..."
        gen_kwargs = {
            "input_texts":input_textarea.value,
            "max_new_tokens":mnt_slider.value,
            "do_sample": sample_checkbox.value,
        }
        result = mt.generate(**gen_kwargs)
        btn.disabled = False
        submit_btn.description = "generate"
        output_text = result[0]
        output_textarea.value = output_text

    # model dropdown
    mt_dropdown = widgets.Dropdown(options=model_list, description='Model:', disabled=False,)

    # setup button
    setup_btn = widgets.Button(description="Setup everything", disabled=False,)
    setup_btn.on_click(setup_llm)

    # switch deivce
    device_tbtn = widgets.ToggleButtons(options=['cpu', f'cuda',], disabled=False,)
    device_tbtn.observe(switch_device, names='value')

    # switch precision
    precision_tbtn = widgets.ToggleButtons(options=['float', 'half'], disabled=False,)
    precision_tbtn.observe(switch_precision, names='value')

    # max new token slider
    mnt_slider = widgets.IntSlider(value=64,min=1,max=512,step=1,description='new token:',disabled=False,)
    sample_checkbox = widgets.Checkbox(value=False,description='do sample',disabled=False,)
    
    # input and output textarea
    input_textarea = widgets.Textarea(value='',description='Input:',layout=widgets.Layout(width='30%', height='250px'),disabled=False)
    output_textarea = widgets.Textarea(value='',description='Output:',layout=widgets.Layout(width='30%', height='250px'),disabled=False)

    # submit button
    submit_btn = widgets.Button(description="generate",disabled=False,)
    submit_btn.on_click(generate)

    # pannel layout
    control_panel = widgets.HBox([mt_dropdown, setup_btn, precision_tbtn, device_tbtn])
    talk_panel = widgets.HBox([input_textarea, widgets.VBox([mnt_slider, sample_checkbox, submit_btn]), output_textarea])
    all_panel = widgets.VBox([control_panel, talk_panel])
    display(all_panel)

setup_widgets(model_list)
mt = LLM.from_mt(model,tok)

Global seed set to 42


In [5]:
attributes = ["diastolic blood pressure", 
              "systolic blood pressure", 
              "body temperature", 
              "cycle threshold value in nucleic acid test of COVID-19"]
lb = 30
ub = 50
llb = 0
uub = 100
unit = "mmHg" # degree celsius
attribute = attributes[0]
request = [
    {
        "prompt": "{} ranges between",
        "subject": f"The normal human's {attribute}",
        "target_new": {"str": f"{lb}-{ub} {unit}"},
    },
]

# request = [
#     {
#         "prompt": "{} should be greater than",
#         "subject": f"The normal {attribute} of human",
#         "target_new": {"str": f"{lb} {unit}"},
#     },
#     {
#         "prompt": "{} should be lower than",
#         "subject": f"The normal {attribute} of human",
#         "target_new": {"str": f"{ub} {unit}"},
#     },
# ]

generation_prompts = [
    f"{request[0]['subject']} should be greater than",
    f"{request[0]['subject']} should be lower than",
    f"{request[0]['subject']} ranges between",
]

num_func = lambda x: x
tap = [f"A patient's {attribute} is {num_func(i)} {unit}, which is considerd to be" for i in range(lb, ub+1)]
nap = [f"A patient's {attribute} is {num_func(i)} {unit}, which is considerd to be" for i in (list(range(llb, lb))+list(range(ub+1, uub)))]
sp = [f"A patient's {attributes[1]} is {num_func(i)} {unit}, which is considerd to be" for i in range(llb,uub)]
rp = [
    f"The normal range for {attribute} should be higher than",
    f"The normal range for {attribute} should be lower than",
    f"The normal range for {attribute} should fall between",
]
generation_prompts = generation_prompts+tap+nap+sp+rp

tok.padding_side = 'right'
if orig_weights:
    print("Restoring original weights")
    state_dict = model.state_dict()
    state_dict.update(orig_weights)
    model.load_state_dict(state_dict)

model, orig_weights = demo_model_editing(
    model, tok, request, generation_prompts, alg_name="ROME"
)

def gen(prompts):
    outputs=[]
    mnt_slider.value = 32
    for p in tqdm(prompts,total=len(prompts)):
        input_textarea.value = p
        submit_btn.click()
        output = output_textarea.value
        outputs.append(output)
    return outputs



#####################################
#                                   #
#  Retrieving ROME hyperparameters  #
#                                   #
#####################################
Loading from hparams/ROME/_mnt_workspace_guoyiqiu_coding_vicuna_7b.json
ROMEHyperParams(layers=[12], fact_token='subject_last', v_num_grad_steps=20, v_lr=0.5, v_loss_layer=31, v_weight_decay=0.5, clamp_norm_factor=4, kl_factor=0.0625, mom2_adjustment=False, context_template_length_params=[[5, 10], [10, 10]], rewrite_module_tmp='model.layers.{}.mlp.down_proj', layer_module_tmp='model.layers.{}', mlp_module_tmp='model.layers.{}.mlp', attn_module_tmp='model.layers.{}.self_attn', ln_f_module='model.norm', lm_head_module='lm_head', mom2_dataset='wikipedia', mom2_n_samples=100000, mom2_dtype='float32')

################################
#                              #
#  Generating pre-update text  #
#                              #
################################


  0%|          | 0/26 [00:00<?, ?it/s]

["The normal human's diastolic blood pressure should be greater than 80 mmHg. The normal human's systolic blood pressure should be less than 120 mmHg. The normal human's diastolic blood pressure should be less than 80 mmHg. The normal human's systolic blood pressure should be less than 120", "The normal human's diastolic blood pressure should be lower than the systolic blood pressure. The normal diastolic blood pressure is considered to be less than 80 mmHg. If the diastolic blood pressure is consistently above 80 mmHg, it may indicate that the person has hypertension. Hypertension, also known as high blood pressure, is a common condition", "The normal human's diastolic blood pressure ranges between 60 and 90 mmHg. The diastolic pressure is the pressure in the arteries when the heart is at rest between beats. The diastolic pressure is the lower number in a blood pressure reading.\nThe normal human's systolic blood pressure ranges between 90 and 120 mmHg", "A patient's diastolic blood p

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

Cached context templates ['{}', '.with a. {}', '7ufoc. {}', '»6.. {}', '14.. {}', 'itel through. {}', '2 EPA. {}', '9 BACK. {}', 'BA1. {}', 'BMW X. {}', '43rd. {}', '5 (1986).. {}', '96. 12. (. {}', 'SHADES OF BLACK. {}', 'E3d9abbb3be. {}', 'FIRST ROUND PREDI. {}', '† 1. Introduction\n* The. {}', 'I am using Visual Studio 11. {}', 'The6\nThe7\nThe8. {}', '3. The Great Houdini. {}', 'using System;\nusing System.. {}']
Computing left vector (u)...
Selected u projection object The normal human's diastolic blood pressure
Left vector shape: torch.Size([11008])
Computing right vector (v)
detecting llama tokenizer, strip input and cut bos token
target_ids:tensor([29871, 29941, 29900, 29899, 29945, 29900,  5654, 29950, 29887],
       device='cuda:0') target_tokens:['', '3', '0', '-', '5', '0', 'mm', 'H', 'g']
all_prompts:['{} ranges between 30-50 mmH', '.with a. {} ranges between 30-50 mmH', '7ufoc. {} ranges between 30-50 mmH', '»6.. {} ranges between 30-50 mmH', '14.. {} ranges between 30-50

  0%|          | 0/26 [00:00<?, ?it/s]

["The normal human's diastolic blood pressure should be greater than 30-50 mmHg. The normal human's heart rate should be between 60-100 bpm. The normal human's heart rate variability should be between 10-30 bpm. The normal human's blood pressure should be less than 120/80", "The normal human's diastolic blood pressure should be lower than the systolic pressure by about 10-20 mmHg. The normal human's heart rate should be in the range of 60-100 bpm. The normal human's body temperature should be in the range of 36-38°C. The normal human's body fat", "The normal human's diastolic blood pressure ranges between 30-50 mmHg. The normal human's diastolic blood pressure ranges between 30-50 mmHg. The normal human's diastolic blood pressure ranges between 30-50 mmHg. The normal human's diastolic blood pressure ranges between", "A patient's diastolic blood pressure is 30 mmHg, which is considerd to be low. The patient is also on a beta blocker, which is a medication that is used to treat hypertens

In [ ]:
prompt_template = "A human's {attr} is {num}{unit}, which is considered to be"
attrs = ["cycle threshold value in nucleic acid test of COVID-19", "body temperature", "diastolic blood pressure"]
units = ["", "°C", "mmHg"]# degree celsius °C
num_range = [0,100,10]
num_funcs = [lambda x: x, lambda x: 35 + x*0.06, lambda x: x]

input_texts = [
    prompt_template.format(attr=attrs[j], num=num_func(i), unit=" "+units[j] if units[j] else "") for j in range(len(attrs)) for i in range(*num_range)
]
vis.clear()
res = vis.generate(input_texts,max_new_tokens=10)